In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Ensemble classification with Rigetti and Qiskit devices
=======================================================

::: {.meta}
:property=\"og:description\": We demonstrate how two QPUs can be
combined in parallel to help solve a machine learning classification
problem, using PyTorch and PennyLane. :property=\"og:image\":
<https://pennylane.ai/qml/_images/ensemble_diagram.png>
:::

*Author: Tom Bromley --- Posted: 14 February 2020. Last updated: 13
December 2021.*

This tutorial outlines how two QPUs can be combined in parallel to help
solve a machine learning classification problem.

We use the `rigetti.qvm` device to simulate one QPU and the `qiskit.aer`
device to simulate another. Each QPU makes an independent prediction,
and an ensemble model is formed by choosing the prediction of the most
confident QPU. The iris dataset is used in this tutorial, consisting of
three classes of iris flower. Using a pre-trained model and the PyTorch
interface, we\'ll see that ensembling allows the QPUs to specialize
towards different classes.

Let\'s begin by importing the prerequisite libraries:


This tutorial requires the `pennylane-rigetti` and `pennylane-qiskit`
packages, which can be installed by following the instructions
[here](https://pennylane.ai/install.html). We also make use of the
[PyTorch interface
\<https://pennylane.readthedocs.io/en/stable/introduction
/interfaces.html\>](), which can be installed from
[here](https://pytorch.org/get-started/locally/).

Load data
=========

The next step is to load the iris dataset.


We shuffle the data and then embed the four features into a
two-dimensional space for ease of plotting later on. The first two
principal components of the data are used.


We will be encoding these two features into quantum circuits using
`~.pennylane.RX`{.interpreted-text role="class"} rotations, and hence
renormalize our features to be between $[-\pi, \pi]$.


The data is split between a training and a test set. This tutorial uses
a model that is pre-trained on the training set.


Finally, let\'s take a quick look at our data:


![](/demonstrations/ensemble_multi_qpu/ensemble_multi_qpu_001.png){.align-center
width="80.0%"}


This plot shows us that class 0 points can be nicely separated, but that
there is an overlap between points from classes 1 and 2.

Define model
============

Our model is summarized in the figure below. We use two 4-qubit devices:
`4q-qvm` from the pennyLane-rigetti plugin and `qiskit.aer` from the
PennyLane-Qiskit plugin.

Data is input using `~.pennylane.RX`{.interpreted-text role="class"}
rotations and then a different circuit is enacted for each device with a
unique set of trainable parameters. The output of both circuits is a
`~.pennylane.PauliZ`{.interpreted-text role="class"} measurement on
three of the qubits. This is then fed through a softmax function,
resulting in two 3-dimensional probability vectors corresponding to the
3 classes.

Finally, the ensemble model chooses the QPU which is most confident
about its prediction (i.e., the class with the highest overall
probability over all QPUs) and uses that to make a prediction.

![](/demonstrations/ensemble_multi_qpu/ensemble_diagram.png){.align-center
width="80.0%"}

Quantum nodes
-------------

We begin by defining the two quantum devices and the circuits to be run
on them.


::: {.note}
::: {.title}
Note
:::

If you have access to Rigetti hardware, you can swap out `rigetti.qvm`
for `rigetti.qpu` and specify the hardware device to run on. Users with
access to the IBM Q Experience can swap `qiskit.aer` for `qiskit.ibmq`
and specify their chosen backend (see
[here](https://pennylane-qiskit.readthedocs.io/en/latest/gettingstarted.html#ibm-q-experience)).
:::

::: {.warning}
::: {.title}
Warning
:::

Rigetti\'s QVM and Quil Compiler services must be running for this
tutorial to execute. They can be installed by consulting the [Rigetti
documentation](http://docs.rigetti.com/qcs/) or, for users with Docker,
by running:

``` {.bash}
docker run -d -p 5555:5555 rigetti/quilc -R -p 5555
docker run -d -p 5000:5000 rigetti/qvm -S -p 5000
```
:::

The circuits for both QPUs are shown in the figure below:

![](/demonstrations/ensemble_multi_qpu/diagram_circuits.png){.align-center
width="80.0%"}


We finally combine the two devices into a
`~.pennylane.QNode`{.interpreted-text role="class"} list that uses the
PyTorch interface:


Postprocessing into a prediction
================================

The `predict_point` function below allows us to find the ensemble
prediction, as well as keeping track of the individual predictions from
each QPU.

We include a `parallel` keyword argument for evaluating the
`~.pennylane.QNode`{.interpreted-text role="class"} list in a parallel
asynchronous manner. This feature requires the `dask` library, which can
be installed using `pip install "dask[delayed]"`. When `parallel=True`,
we are able to make predictions faster because we do not need to wait
for one QPU to output before running on the other.


Next, let\'s define a function to make a predictions over multiple data
points.


Make predictions
================

To test our model, we first load a pre-trained set of parameters which
can also be downloaded by clicking
`here <../demonstrations/ensemble_multi_qpu/params.npy>`{.interpreted-text
role="download"}.


We can then make predictions for the training and test datasets.


::: {.rst-class}
sphx-glr-script-out

Out:

``` {.none}
Predicting on training dataset
Completed up to iteration 10
Completed up to iteration 20
Completed up to iteration 30
Completed up to iteration 40
Completed up to iteration 50
Completed up to iteration 60
Completed up to iteration 70
Completed up to iteration 80
Completed up to iteration 90
Completed up to iteration 100
Completed up to iteration 110
Completed up to iteration 120
Predicting on test dataset
Completed up to iteration 10
Completed up to iteration 20
```
:::


Analyze performance
===================

The last thing to do is test how well the model performs. We begin by
looking at the accuracy.

Accuracy
--------


::: {.rst-class}
sphx-glr-script-out

Out:

``` {.none}
Training accuracy (ensemble): 0.824
Training accuracy (QPU0):  0.648
Training accuracy (QPU1):  0.296
```
:::


::: {.rst-class}
sphx-glr-script-out

Out:

``` {.none}
Test accuracy (ensemble): 0.72
Test accuracy (QPU0):  0.56
Test accuracy (QPU1):  0.24
```
:::


These numbers tell us a few things:

-   On both training and test datasets, the ensemble model outperforms
    the predictions from each QPU. This provides a nice example of how
    QPUs can be used in parallel to gain a performance advantage.
-   The accuracy of QPU0 is much higher than the accuracy of QPU1. This
    does not mean that one device is intrinsically better than the
    other. In fact, another set of parameters can lead to QPU1 becoming
    more accurate. We will see in the next section that the difference
    in accuracy is due to specialization of each QPU, which leads to
    overall better performance of the ensemble model.
-   The test accuracy is lower than the training accuracy. Here our
    focus is on analyzing the performance of the ensemble model, rather
    than minimizing the generalization error.

Choice of QPU
=============

Is there a link between the class of a datapoint and the QPU chosen to
make the prediction in the ensemble model? Let\'s investigate.


In [ ]:
# Combine choices_train and choices_test to simplify analysis

::: {.rst-class}
sphx-glr-script-out

Out:

``` {.none}
Choices: [0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 0 1 0 0 1 0 1 0 0 0
 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0
 0 0]
Choices counts: Counter({0: 110, 1: 40})
```
:::


The following lines keep track of choices and corresponding predictions
in the ensemble model.


We can hence find the predictions each QPU was responsible for.


::: {.rst-class}
sphx-glr-script-out

Out:

``` {.none}
When QPU0 was chosen by the ensemble, it made the following distribution of predictions:
Counter({0: 55, 2: 55})

When QPU1 was chosen by the ensemble, it made the following distribution of predictions:
Counter({1: 37, 0: 3})

Distribution of classes in iris dataset: Counter({0: 50, 2: 50, 1: 50})
```
:::


These results show us that QPU0 specializes to making predictions on
classes 0 and 2, while QPU1 specializes to class 1.

Visualization
=============

We conclude by visualizing the correct and incorrect predictions on the
dataset. The following function plots correctly predicted points in
green and incorrectly predicted points in red.


We can again compare the ensemble model with the individual models from
each QPU.


![](/demonstrations/ensemble_multi_qpu/ensemble_multi_qpu_002.png){.align-center
width="80.0%"}


![](/demonstrations/ensemble_multi_qpu/ensemble_multi_qpu_003.png){.align-center
width="80.0%"}


![](/demonstrations/ensemble_multi_qpu/ensemble_multi_qpu_004.png){.align-center
width="80.0%"}


These plots reinforce the specialization of the two QPUs. QPU1
concentrates on doing a good job at predicting class 1, while QPU0 is
focused on classes 0 and 2. By combining together, the resultant
ensemble performs better.

This tutorial shows how QPUs can work in parallel to realize a
performance advantage. Check out our `vqe_parallel`{.interpreted-text
role="doc"} tutorial to see how multiple QPUs can be evaluated
asynchronously to speed up calculating the potential energy surface of
molecular hydrogen!


About the author
================
